In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [60]:
t1= "buena idea"
t2 = "terrible propuesta"
t3 = "buena propuesta"
t4 = "terrible idea"
t5 = "buena propuesta pero tarde"

xtrain = [t1,t2,t3,t4,t5]
ytrain = ["positivo", "negativo", "positivo", "negativo", "negativo"]

In [62]:
xpred = ["buena propuesta"]

#positivo
pY_positivo = 2/5
#p(X|positivo) = p(buena|positivo)*p(propuesta|positivo)
alfa=1
pXY_positivo = ((2+alfa)/(4+6*alfa))*((1+alfa)/(4+6*alfa))
pYX_positivo = pXY_positivo * pY_positivo





#positivo
pY_negativo = 3/5
#p(X|negativo) = p(buena|negativo)*p(propuesta|negativo)
alfa=1
pXY_negativo = ((1+alfa)/(8+6*alfa))*((2+alfa)/(8+6*alfa))
pYX_negativo = pXY_negativo * pY_negativo
pYX_negativo

total = pXY_negativo+pXY_positivo
print(pYX_positivo/total)
print(pYX_negativo/total)

0.2648648648648649
0.20270270270270271


In [69]:
class BayesIngenuo():
  def __init__(self):
    self.vocabulario = None
    self.clases = None
    self.pY=None
    self.pXY=None

  def CrearVocabulario(self, xtrain):
    vocabulario = set()
    for x in xtrain:
      for t in x.split():
        vocabulario.add(t)
    self.vocabulario = list(vocabulario)
    return self.vocabulario

  def CrearClases(self, ytrain):
    clases = set()
    for y in ytrain:
      clases.add(y)
    self.clases = list(clases)
    return self.clases

  def CalcularPY(self, ytrain):
    pY = np.zeros(len(self.clases))
    for c in ytrain:
      idy = self.clases.index(c)
      pY[idy] += 1/len(ytrain)
    self.pY = pY
    return pY
  def fit(self, xtrain, ytrain, alfa=1):
    self.CrearVocabulario(xtrain)
    self.CrearClases(ytrain)
    self.CalcularPY(ytrain)
    pXY = np.zeros(shape=(len(self.vocabulario), len(self.clases)))
    ny = pXY.copy()
    for x, y in zip(xtrain, ytrain):
      idy = self.clases.index(y)
      for t in x.split():
        idx = self.vocabulario.index(t)
        ny[idx,idy]+=1

    for idx in range(len(self.vocabulario)):
      for idy in range(len(self.clases)):
        pXY[idx,idy] = \
        (ny[idx,idy]+alfa)/(ny[:,idy].sum() + alfa*len(self.vocabulario))
    self.pXY = pXY

  def predict_proba(self, x):
    prob = np.ones(len(self.clases))
    for idy in range(len(self.clases)):
      for t in x.split():
        idx = self.vocabulario.index(t)
        prob[idy] = self.pXY[idx,idy]*prob[idy]
      prob[idy]=prob[idy]*self.pY[idy]
    return prob/prob.sum()

In [70]:
bayes = BayesIngenuo()

In [71]:
bayes.fit(xtrain,ytrain)
print(bayes.pY)
print(bayes.clases)
print(bayes.pXY)
print(bayes.predict_proba("buena propuesta"))

[0.4 0.6]
['positivo', 'negativo']
[[0.1        0.21428571]
 [0.1        0.14285714]
 [0.2        0.21428571]
 [0.3        0.14285714]
 [0.2        0.14285714]
 [0.1        0.14285714]]
[0.56647399 0.43352601]


In [72]:
import re

import sklearn.preprocessing as skp
import sklearn.model_selection as skms
import sklearn.feature_extraction.text as sktext
import sklearn.naive_bayes as skbayes

from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import SnowballStemmer

In [75]:
xencoder = sktext.CountVectorizer()
xencoder.fit(xtrain)
pd.DataFrame(
    xencoder.transform(xtrain).todense(),
    columns = xencoder.get_feature_names()
)

,buena,idea,pero,propuesta,tarde,terrible
0,1,1,0,0,0,0
1,0,0,0,1,0,1
2,1,0,0,1,0,0
3,0,1,0,0,0,1
4,1,0,1,1,1,0


In [76]:
yencoder = skp.LabelEncoder()
yencoder.fit(ytrain)
yencoder.inverse_transform([0,1])

array(['negativo', 'positivo'], dtype='<U8')

In [77]:
modelo = skbayes.MultinomialNB()
modelo.fit(xencoder.transform(xtrain), yencoder.transform(ytrain))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [78]:
modelo.predict_proba(xencoder.transform(["buena propuesta"]))

array([[0.43352601, 0.56647399]])

# ejemplo analisis de sentimiento con twitter

In [79]:
!wget lalussoft.com/mldemos/tweets1.xls

--2021-07-16 02:52:14--  http://lalussoft.com/mldemos/tweets1.xls
Resolving lalussoft.com (lalussoft.com)... 34.198.109.148
Connecting to lalussoft.com (lalussoft.com)|34.198.109.148|:80... connected.
HTTP request sent, awaiting response... 302 Redirect
Location: https://lalussoft.com/mldemos/tweets1.xls [following]
--2021-07-16 02:52:14--  https://lalussoft.com/mldemos/tweets1.xls
Connecting to lalussoft.com (lalussoft.com)|34.198.109.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 917504 (896K) [application/vnd.ms-excel]
Saving to: ‘tweets1.xls’

tweets1.xls         100%[===================>] 896.00K  2.35MB/s    in 0.4s    

2021-07-16 02:52:15 (2.35 MB/s) - ‘tweets1.xls’ saved [917504/917504]



In [80]:
tweets = pd.read_excel("tweets1.xls")
tweets.head()

,tweet,class
0,abcdesevilla.es: Recio no tiene «indicios pote...,neg
1,abcdesevilla.es: Cuatro altos cargos de Empleo...,neg
2,La marcha atrás del PP en posponer devolución ...,neg
3,Accidente en BUS-VAO A-6 km. 12. Motorista de ...,neg
4,"#FF a ti, que deseas desesperadamente hacerme ...",neg


In [81]:
xtrain, xtest, ytrain, ytest = skms.train_test_split(tweets.tweet, tweets["class"], test_size=0.3, random_state=0)


In [82]:
tokenizer = TweetTokenizer()
Stemmer = SnowballStemmer(language="spanish")
usar_stemmer = 0
def TokenizarTweet(s):
  #eliminamos la URL (falta arreglar)
  s = re.sub(r"(http://|https://)[\w\d_-]+(\.[\w\d_-])*\.\w{2,}(/[\w\d\_+#.-]+)*","",s)

  #eliminar hashtag
  s = re.sub(r"#[\w\dáéíóúÁÉÍÓÚñ_]+", "",s)

  #eliminar @xx
  s = re.sub(r"@[\w\dáéíóúÁÉÍÓÚñ_]+", "",s)

  #eliminar otros
  s = re.sub(r"[¡]", "",s)
  s = re.sub(r"[¿]", "",s)

  #uniformizar cantidades
  s = re.sub(r"\s\d+[.]?\d+|\s[.]\d+","CANTIDAD",s)
  s = re.sub(r"\s\d+([,]\d+)?","CANTIDAD",s)

  #otras expresiones no relevantes
  s = s.replace("»", " ")
  s = s.replace("«", " ")
  s = s.replace("\"", " ")

  tokens = tokenizer.tokenize(s)

  #aplicar stop words
  stop_list = ["a", "e", "i", "o", "u", "y", "en", "la", "al", "un", "una", "de", "el", "le",
               "su", "los", ".", ",", "*", "(", ")", "&"]
  tokens = [token for token in tokens if token not in stop_list]
  if usar_stemmer==1:
    tokens = [Stemmer.stem(token) for token in tokens]
  return tokens

In [84]:
xencoder = sktext.CountVectorizer(tokenizer=TokenizarTweet)
xencoder.fit(xtrain)
yencoder = skp.LabelEncoder()
yencoder.fit(ytrain)
modelo = skbayes.MultinomialNB()
modelo.fit(xencoder.transform(xtrain), yencoder.transform(ytrain))
modelo.score(xencoder.transform(xtest), yencoder.transform(ytest))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


0.8006578947368421

In [85]:
import requests
import json

In [101]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAJNYRgEAAAAA3CrLvdg6BEwE2aIokBYh6vSOPKM%3DbM9mu9R1s7o0ifnnFA5K8RtwaHOjBh4zSN7myX0H7OhoGJpAzj"
def bearer_oauth(req):
  req.headers["Authorization"] = "Bearer {}".format(bearer_token)
  return req

query_params = {
    'query':'from:SeleccionPeru',
    'max_results':10,
    'tweet.fields':['conversation_id']
}

response = requests.request(
    "GET",
    "https://api.twitter.com/2/tweets/search/recent",
    params=query_params,
    auth=bearer_oauth
    )

if response.status_code != 200:
  print("Error:",response.text)
else:
  j = response.json()

In [102]:
j

{'data': [{'conversation_id': '1415862736991539204',
   'id': '1415862736991539204',
   'text': '¡Conferencia de prensa! 🎙\n\nEste viernes, desde las 11:30 horas, no te pierdas la palabra de Ricardo Gareca, entrenador de nuestra @SeleccionPeru 🇵🇪. \n\n▶️ Síguela en vivo a través de nuestro canal de Youtube y #FPFPlay https://t.co/TET6WbHKcI'},
  {'conversation_id': '1415834456318005249',
   'id': '1415834456318005249',
   'text': '¡Trabajando en mejorar! 🙌🏻⚽️\n\nLas futbolistas Yamilé Rivas y Ariana Cubas hablaron sobre la importancia del primer microciclo presencial de la @SeleccionPeru Femenina Sub 20 🇵🇪.\n\n#ArribaPerú 💪🏻 https://t.co/7tHA32IUGA'},
  {'conversation_id': '1415709018799198212',
   'id': '1415709018799198212',
   'text': '🔴 #FPFPlayAlDía \n\nHOY a la 1:00 p.m. por #FPFPlay tendremos un programa imperdible con entrevista exclusiva a Antonio García Pye, Gerente de Selecciones - FPF. \n\n✔️ Regístrate GRATIS en: https://t.co/J4YSPElj0y\n\nDescárgalo desde #GooglePlay o #A

In [106]:
x = []
xtwitt = []
for t in j["data"]:
  tt = t["text"].lower()
  if tt.find("futbol")>=0 or tt.find("futbol")>=0:
    x.append(t["text"])
    xtwitt.append(t)

In [107]:
xtwitt

[{'conversation_id': '1415834456318005249',
  'id': '1415834456318005249',
  'text': '¡Trabajando en mejorar! 🙌🏻⚽️\n\nLas futbolistas Yamilé Rivas y Ariana Cubas hablaron sobre la importancia del primer microciclo presencial de la @SeleccionPeru Femenina Sub 20 🇵🇪.\n\n#ArribaPerú 💪🏻 https://t.co/7tHA32IUGA'},
 {'conversation_id': '1415453707257458688',
  'id': '1415453707257458688',
  'text': 'Anunciamos a las futbolistas convocadas para el tercer microciclo de la @SeleccionPeru Femenina Absoluta 🇵🇪.\n\n#ArribaPerú 💪🏻 https://t.co/0zf6DMRGF1'},
 {'conversation_id': '1415125249910259712',
  'id': '1415125249910259712',
  'text': '¡Microciclo exitoso! 🙌🏻⚽️\n\nLos futbolistas Emilio Saba y Marco Huamán dieron detalles de lo que fueron los entrenamientos de la @SeleccionPeru Sub 20 🇵🇪 bajo la dirección de Víctor Reyes.\n\n#ArribaPerú 💪🏻 https://t.co/wIneYE0zzJ'}]

In [108]:
y = modelo.predict(xencoder.transform(x))
yprob = modelo.predict_proba(xencoder.transform(x))
print(y)
print(yprob)

[1 0 1]
[[0.01145205 0.98854795]
 [0.61932291 0.38067709]
 [0.02858581 0.97141419]]


In [109]:
for i,tweet in enumerate(x):
  print("tweet Nro: {}, sentimiento: {} con prob: {:0.2f}".format(
      i+1,
      yencoder.inverse_transform([y[i]])[0],
      yprob[i][y[i]]*100
      )
  )
  print("="*30)
  print(tweet,"\n"*2)

tweet Nro: 1, sentimiento: pos con prob: 98.85
¡Trabajando en mejorar! 🙌🏻⚽️

Las futbolistas Yamilé Rivas y Ariana Cubas hablaron sobre la importancia del primer microciclo presencial de la @SeleccionPeru Femenina Sub 20 🇵🇪.

#ArribaPerú 💪🏻 https://t.co/7tHA32IUGA 


tweet Nro: 2, sentimiento: neg con prob: 61.93
Anunciamos a las futbolistas convocadas para el tercer microciclo de la @SeleccionPeru Femenina Absoluta 🇵🇪.

#ArribaPerú 💪🏻 https://t.co/0zf6DMRGF1 


tweet Nro: 3, sentimiento: pos con prob: 97.14
¡Microciclo exitoso! 🙌🏻⚽️

Los futbolistas Emilio Saba y Marco Huamán dieron detalles de lo que fueron los entrenamientos de la @SeleccionPeru Sub 20 🇵🇪 bajo la dirección de Víctor Reyes.

#ArribaPerú 💪🏻 https://t.co/wIneYE0zzJ 




In [110]:
query_params = {
    'query':'conversation_id:1413676922031198213'
}

response = requests.request(
    "GET",
    "https://api.twitter.com/2/tweets/search/recent",
    params=query_params,
    auth=bearer_oauth
    )

if response.status_code != 200:
  print("Error:",response.text)
else:
  jj = response.json()

In [111]:
jj

{'data': [{'id': '1413691220052819970',
   'text': '@dilfellis @SeleccionPeru @FCFSeleccionCol Ese man juega mucho, honestamente no tenía ni idea que existiera hasta esta copa'},
  {'id': '1413685456152272900',
   'text': '@SeleccionPeru @FCFSeleccionCol Lo mínimo que podía hacer, después de regalar el segundo gol.'},
  {'id': '1413682328661004291',
   'text': '@Lyba31142792 @dilfellis @SeleccionPeru @FCFSeleccionCol Perdimos, pero ganamos un gran delantero'},
  {'id': '1413681729441779714',
   'text': '@SeleccionPeru @FCFSeleccionCol es que es mi papi perdimos pero felices del gol de batidula'},
  {'id': '1413681380207255554',
   'text': '@SeleccionPeru @FCFSeleccionCol 🇵🇪🇵🇪🇵🇪🇵🇪'},
  {'id': '1413681214603595783',
   'text': '@SeleccionPeru @FCFSeleccionCol Como olvidar el día que terminó la argolla. Un cambio histórico, el fín de una era.\n\n¡Acéptalo, Raúl Ruidíaz; Gianluca Lapadula fue más! https://t.co/sUbLWlONE9'},
  {'id': '1413681095711895553',
   'text': '@Jade87205397 @dilfell